In [1]:
topics={ 264014: {'title': "how long is life cycle of flea"}}

In [2]:
from mugi import generate_pseudo_references, run_retriever
from openai import OpenAI
import os
from dotenv import load_dotenv
import os

# Load variables from .env file into environment
load_dotenv('.env')

# Access the environment variable
openai_key = os.getenv("OPENAI_API_KEY")
global client
# openai_key = os.environ.get('OPENAI_KEY')
client = OpenAI(api_key=openai_key)

In [3]:
client

In [4]:
new_topics = generate_pseudo_references('gpt-3.5-turbo-1106',topics, generated_document_num=3)

test


  0%|          | 0/1 [00:00<?, ?it/s]

2024-03-06 18:25:47,285 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-06 18:25:49,902 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-06 18:25:52,242 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 1/1 [00:07<00:00,  7.86s/it]


In [5]:
topics = new_topics

In [13]:
from mugi import run_retriever
import utils
from pyserini.search import LuceneSearcher, get_topics, get_qrels
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
print(topics)

({264014: {'title': 'how long is life cycle of flea', 'gen_cand_gpt35': ['The life cycle of a flea typically lasts around 2-3 months, but it can vary depending on environmental conditions such as temperature and humidity. Fleas undergo complete metamorphosis, including four stages: egg, larva, pupa, and adult. The entire life cycle consists of egg hatching (2 days to 2 weeks), larval development (5 days to several months), pupation (7-10 days to several weeks), and emergence of the adult flea.', 'The life cycle of a flea typically lasts around 2-3 months, although it can vary depending on environmental factors such as temperature and humidity. Fleas go through four distinct stages: egg, larva, pupa, and adult. The entire life cycle can be completed in as little as two weeks under ideal conditions, but may take longer in less favorable environments. Understanding the life cycle of fleas is important for effective control and prevention of infestations.', 'The life cycle of a flea typica

In [21]:
from PyPDF2 import PdfFileReader
from jnius import autoclass

# Import necessary Lucene classes
IndexWriterConfig = autoclass('org.apache.lucene.index.IndexWriterConfig')
StandardAnalyzer = autoclass('org.apache.lucene.analysis.standard.StandardAnalyzer')
IndexWriter = autoclass('org.apache.lucene.index.IndexWriter')
Document = autoclass('org.apache.lucene.document.Document')
Field = autoclass('org.apache.lucene.document.Field')
TextField = autoclass('org.apache.lucene.document.TextField')
File = autoclass('java.io.File')
SimpleFSDirectory = autoclass('org.apache.lucene.store.NIOFSDirectory')
Version = autoclass('org.apache.lucene.util.Version')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PdfFileReader(f)
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            text += page.extractText()
    return text

# Function to index text with Lucene
def index_text_with_lucene(pdf_text, index_path):
    analyzer = StandardAnalyzer()
    config = IndexWriterConfig(analyzer)
    directory = SimpleFSDirectory(File(index_path).toPath())
    writer = IndexWriter(directory, config)

    doc = Document()
    doc.add(TextField("content", pdf_text, Field.StoreField.YES))
    writer.addDocument(doc)

    writer.close()

# Path to the PDF file
pdf_path = "../Data/Lakers_Specification.pdf"

# Path to store Lucene index
index_path = "Data/index"

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Index text with Lucene
index_text_with_lucene(pdf_text, index_path)

# Now you can create a Lucene searcher
searcher2 = LuceneSearcher.from_index(index_path)


JavaException: JVM exception occurred: Lock held by this virtual machine: /Users/adambanga/Documents/EDUCATION/Stanford Masters/Stanford Year 2/Winter Quarter/Natural Language Processing(CS 224N)/CS224n_project/mugi/Data/index/write.lock org.apache.lucene.store.LockObtainFailedException

In [14]:
repetition_ratio=5
articles_num=3
topic, gen_key = topics
for key in topic:
      query = topic[key]['title']
      gen_ref = ' '.join(topic[key]['gen_cand_gpt35'][:articles_num])
      repetition_times = (len(gen_ref)//len(query))//repetition_ratio
      topic[key]['enhanced_query'] = (query + ' ')*repetition_times + gen_ref
# bm25_rank_results = run_retriever(topic, searcher, gen_key, k=100)
# eval nDCG@10
# bm25_rank_score=utils.evaluate_bm25(bm25_rank_results,'dl19-passage')
# print(bm25_rank_score)

2024-03-06 18:31:11,166 Running BM25 on query: how long is life cycle of flea


In [22]:
topic

{264014: {'title': 'how long is life cycle of flea',
  'gen_cand_gpt35': ['The life cycle of a flea typically lasts around 2-3 months, but it can vary depending on environmental conditions such as temperature and humidity. Fleas undergo complete metamorphosis, including four stages: egg, larva, pupa, and adult. The entire life cycle consists of egg hatching (2 days to 2 weeks), larval development (5 days to several months), pupation (7-10 days to several weeks), and emergence of the adult flea.',
   'The life cycle of a flea typically lasts around 2-3 months, although it can vary depending on environmental factors such as temperature and humidity. Fleas go through four distinct stages: egg, larva, pupa, and adult. The entire life cycle can be completed in as little as two weeks under ideal conditions, but may take longer in less favorable environments. Understanding the life cycle of fleas is important for effective control and prevention of infestations.',
   'The life cycle of a flea

In [9]:
from model import get_language_model, get_reranker
reranker_model = get_reranker(model_name = 'sentence-transformers/all-mpnet-base-v2', mode = 'concat')
rerank_result = reranker_model.rerank(bm25_rank_results,'gen_cand_gpt35',100,use_enhanced_query=True)
rerank_score = utils.evalute_dict(rerank_result,'dl19-passage')
print(rerank_score)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: string indices must be integers